In [3]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    db,
)

In [4]:
conn = connections.connect(
  alias="default",
  user='root',
  password='Milvus',
  host='localhost',
  port='19530'
)

In [5]:
database = db.create_database("gaming")
db.using_database("gaming")

### Create the memory_stream collection

In [7]:
memory_id = FieldSchema(
    name="id",
    dtype=DataType.INT64,
    is_primary=True,
)

# suppose there is a MySQL database, maintaining information about the character, including their state and attributes
# this field records the primary key of character
character_id = FieldSchema(
    name="character_id",
    dtype=DataType.INT64
)

importance = FieldSchema(
  name="importance",
  dtype=DataType.INT8,
)

create_time = FieldSchema(
  name="create_time",
  dtype=DataType.VARCHAR,
  max_length=50,
)

# three types: observation, reflection and plan
# this is the field used to partition
category = FieldSchema(
  name="category",
  dtype=DataType.VARCHAR,
  max_length=50,
)

vector = FieldSchema(
  name="memory_vector",
  dtype=DataType.FLOAT_VECTOR,  # BINARY_VECTOR or FLOAT_VECTOR
  default_value=[[1.0] * 8848],
  dim=8848
)

content = FieldSchema(
    name="real_memory",
    dtype=DataType.VARCHAR,
    max_length=9999,
    default_value="Unknown"
)

In [8]:
coll_schema = CollectionSchema(
  fields=[memory_id, character_id, importance, create_time, category, vector, content],
  description="memory",
  
  # 不开启 dynamic 意味着之后插入的所有数据实体都要与 schema 匹配；而开启后，可以插入具有新字段的实体，有点类似 nonsql
  # https://milvus.io/docs/dynamic_schema.md#Create-collection-with-dynamic-schema-enabled
  enable_dynamic_field=True
)

collection_name = "character_memory"

collection = Collection(
    name=collection_name,
    schema=coll_schema,
    using='default',  # 在哪个 server 上创建 collection
    shards_num=1,  # 把写入操作分配到 2 个不同的 node / server 上并行进行
)

In [11]:
# create the partitions

collection = Collection("character_memory")    
collection.create_partition("observation")
collection.create_partition("reflection")
collection.create_partition("long_term_plan")

{"name":"long_term_plan","collection_name":"character_memory","description":""}

### create the chat_dialog collection

In [9]:
dialog_id = FieldSchema(
    name="id",
    dtype=DataType.INT64,
    is_primary=True,
)

# the primary key of speaking character
source_id = FieldSchema(
    name="source_id",
    dtype=DataType.INT64
)

target_id = FieldSchema(
  name="target_id",
    dtype=DataType.INT64
)

create_time = FieldSchema(
  name="create_time",
  dtype=DataType.VARCHAR,
  max_length=50,
)

vector = FieldSchema(
  name="dialog_vector",
  dtype=DataType.FLOAT_VECTOR,  # BINARY_VECTOR or FLOAT_VECTOR
  default_value="Unknown",
  dim=8848
)

content = FieldSchema(
    name="real_dialog",
    dtype=DataType.VARCHAR,
    max_length=9999,
    default_value="Unknown"
)

In [10]:
coll_schema = CollectionSchema(
  fields=[dialog_id, source_id, create_time, target_id, vector, content],
  description="dialog",
  
  # 不开启 dynamic 意味着之后插入的所有数据实体都要与 schema 匹配；而开启后，可以插入具有新字段的实体，有点类似 nonsql
  # https://milvus.io/docs/dynamic_schema.md#Create-collection-with-dynamic-schema-enabled
  enable_dynamic_field=True
)

collection_name = "dialog_history"


collection = Collection(
    name=collection_name,
    schema=coll_schema,
    using='default',  # 在哪个 server 上创建 collection
    shards_num=1  # 把写入操作分配到 2 个不同的 node / server 上并行进行
)

{"name":"observation","collection_name":"character_memory","description":""}

In [64]:
data = [
  [i for i in range(20)],
  [i for i in range(20)],
  [2 for i in range(20)],
  ["2029-1-1" for _ in range(20)],
  ["obser" for _ in range(20)],
  [[1.0] * 8848 for _ in range(20)],
  ["aaaa" for _ in range(20)]
]

In [65]:
collection.insert(data, 'observation') 

(insert count: 20, delete count: 0, upsert count: 0, timestamp: 447111624051392515, success count: 20, err count: 0)

In [66]:
from pymilvus import Collection, utility

index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":2}
}

collection = Collection("character_memory")      
collection.create_index(
  field_name="memory_vector", 
  index_params=index_params
)

utility.index_building_progress("character_memory")

{'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0}

In [67]:
from pymilvus import Partition

partition = Partition("character_memory", "observation")       # Get an existing partition.
partition.load(replica_number=1)